In [ ]:
!pip install -U transformers

In [ ]:
# Basic medical VQA with MedGemma
from transformers import pipeline
from PIL import Image
import torch
import os
from pathlib import Path

# Initialize pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Find image file with robust path handling
possible_paths = [
    "../../data/xray.jpg",  # From tools subdirectory
    "../data/xray.jpg",     # From src directory
    "data/xray.jpg",        # From project root
    "XrayAgent/data/xray.jpg"  # If running from parent directory
]

image_path = None
for path in possible_paths:
    if os.path.exists(path):
        image_path = path
        break

if image_path is None:
    raise FileNotFoundError("Could not find xray.jpg. Please ensure the image exists in the data directory.")

image = Image.open(image_path)

questions = ["Is there evidence of pneumonia in this image?"]

for question in questions:
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are an expert radiologist. Provide detailed, accurate answers to medical questions about images."}]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": question}, {"type": "image", "image": image}]
        }
    ]
    output = pipe(text=messages, max_new_tokens=200)
    answer = output[0]["generated_text"][-1]["content"]
    print(f"Q: {question}")
    print(f"A: {answer}")

/Users/zxm/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
